In [ ]:
"""
trade_monitor_module.py

Trade monitor + trade journal (backtracking) + Prowl notifications.

Belangrijk:
- Dit script start de monitor NIET automatisch.
- Gebruik vanuit je notebook:
    from trade_monitor_module import start_monitor, stop_monitor, manual_close_open_trade

Database:
- Maakt automatisch deze tabellen aan als ze niet bestaan:
    - monitor_state
    - trade_journal

Data input:
- Probeert automatisch een tabel met (timestamp, price) te detecteren.
- Preference: gold_spot_prices(ts_utc, price_per_troy_oz)
- Fallback: tick/price/quotes tabellen met herkenbare kolomnamen.

Dependencies:
    pip install pandas requests
"""

from __future__ import annotations

from dataclasses import dataclass
from datetime import datetime, timezone
from typing import Optional, Tuple, Dict, Any
import os
import sqlite3
import threading
import time

import pandas as pd
import requests


# =========================
# CONFIG / DATACLASSES
# =========================

@dataclass
class StrategyConfig:
    # Bollinger
    bb_period: int = 20
    bb_std: float = 2.0

    # EMA
    ema_fast: int = 20
    ema_slow: int = 50

    # MACD
    macd_fast: int = 12
    macd_slow: int = 26
    macd_signal: int = 9

    # Trade plan
    rr: float = 1.5               # TP = Entry + rr * risk (LONG), omgekeerd voor SHORT
    swing_lookback: int = 8       # bars voor swing-high/low
    sl_buffer: float = 0.5        # buffer in prijs-eenheden


@dataclass
class TradePlan:
    side: str          # "LONG" of "SHORT"
    entry: float
    sl: float
    tp: float
    rationale: str

    def plan_id(self) -> str:
        return f"{self.side}|{self.entry:.5f}|{self.sl:.5f}|{self.tp:.5f}"


@dataclass
class AdviceResult:
    label: str
    message: str
    ts_utc: str
    plan: Optional[TradePlan] = None


# =========================
# DB HELPERS
# =========================

def db_path_from_env(default: str = "prices.sqlite") -> str:
    return os.environ.get("DB_PATH", default)


def _detect_ticks_table_and_columns(conn: sqlite3.Connection) -> Tuple[str, str, str]:
    cur = conn.cursor()
    tables = [r[0] for r in cur.execute(
        "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
    ).fetchall()]

    # 1) Hard preference: jouw collector tabel (gold)
    if "gold_spot_prices" in tables:
        cols = [r[1] for r in cur.execute("PRAGMA table_info(gold_spot_prices)").fetchall()]
        if "ts_utc" in cols and "price_per_troy_oz" in cols:
            return "gold_spot_prices", "ts_utc", "price_per_troy_oz"

    # 2) Generiek fallback
    preferred_tables = ["ticks", "tick", "prices", "price", "quotes", "quote"]
    table_candidates = preferred_tables + [t for t in tables if t not in preferred_tables]

    ts_candidates = ["ts", "timestamp", "time", "datetime", "date", "ts_utc"]
    price_candidates = ["price", "last", "close", "bid", "ask", "mid", "price_per_troy_oz", "price_per_gram"]

    for table in table_candidates:
        if table not in tables:
            continue
        cols = [r[1] for r in cur.execute(f"PRAGMA table_info({table})").fetchall()]
        ts_col = next((c for c in ts_candidates if c in cols), None)
        price_col = next((c for c in price_candidates if c in cols), None)
        if ts_col and price_col:
            return table, ts_col, price_col

    raise RuntimeError(
        "Kon geen bron detecteren. Verwacht een tabel met timestamp + price.\n"
        "Voorbeeld: gold_spot_prices(ts_utc, price_per_troy_oz) of ticks(ts, price)."
    )


def _parse_ts_to_datetime_utc(series: pd.Series) -> pd.DatetimeIndex:
    s = series.copy()

    if s.dtype == object:
        dt = pd.to_datetime(s, utc=True, errors="coerce")
        if dt.notna().any():
            return pd.DatetimeIndex(dt)

    s_num = pd.to_numeric(s, errors="coerce")
    if s_num.dropna().empty:
        dt = pd.to_datetime(s, utc=True, errors="coerce")
        return pd.DatetimeIndex(dt)

    # Heuristiek: > 10^12 => ms
    if s_num.dropna().median() > 1_000_000_000_000:
        dt = pd.to_datetime(s_num, unit="ms", utc=True, errors="coerce")
    else:
        dt = pd.to_datetime(s_num, unit="s", utc=True, errors="coerce")

    return pd.DatetimeIndex(dt)


def load_prices_from_sqlite(db_path: str, limit_rows: int = 20000) -> pd.DataFrame:
    conn = sqlite3.connect(db_path)
    try:
        table, ts_col, price_col = _detect_ticks_table_and_columns(conn)
        query = f"""
            SELECT {ts_col} AS ts, {price_col} AS price
            FROM {table}
            ORDER BY {ts_col} DESC
            LIMIT ?
        """
        df = pd.read_sql_query(query, conn, params=(limit_rows,))
        if df.empty:
            return df

        df["ts"] = _parse_ts_to_datetime_utc(df["ts"])
        df["price"] = pd.to_numeric(df["price"], errors="coerce")
        df = df.dropna(subset=["ts", "price"]).sort_values("ts")
        return df
    finally:
        conn.close()


def prices_to_ohlc(prices: pd.DataFrame, timeframe: str = "1min") -> pd.DataFrame:
    if prices.empty:
        return pd.DataFrame()

    df = prices.copy().set_index("ts").sort_index()
    ohlc = df["price"].resample(timeframe).ohlc().dropna()
    return ohlc


# =========================
# INDICATORS
# =========================

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()


def bollinger(close: pd.Series, period: int, std: float):
    mid = close.rolling(period).mean()
    sd = close.rolling(period).std(ddof=0)
    upper = mid + std * sd
    lower = mid - std * sd
    return mid, upper, lower


def macd(close: pd.Series, fast: int, slow: int, signal: int):
    macd_line = ema(close, fast) - ema(close, slow)
    signal_line = ema(macd_line, signal)
    hist = macd_line - signal_line
    return macd_line, signal_line, hist


def compute_indicators(ohlc: pd.DataFrame, cfg: StrategyConfig) -> pd.DataFrame:
    df = ohlc.copy()
    df["ema_fast"] = ema(df["close"], cfg.ema_fast)
    df["ema_slow"] = ema(df["close"], cfg.ema_slow)
    df["bb_mid"], df["bb_upper"], df["bb_lower"] = bollinger(df["close"], cfg.bb_period, cfg.bb_std)
    df["macd"], df["macd_signal"], df["macd_hist"] = macd(df["close"], cfg.macd_fast, cfg.macd_slow, cfg.macd_signal)
    return df


# =========================
# LABELING (SETUPS)
# =========================

def label_latest(df: pd.DataFrame, cfg: StrategyConfig) -> str:
    if df.empty or len(df) < max(cfg.bb_period, cfg.ema_slow, cfg.macd_slow) + 5:
        return "NO_DATA"

    cur = df.iloc[-1]
    prev = df.iloc[-2]
    prev2 = df.iloc[-3]

    uptrend = (cur["ema_fast"] > cur["ema_slow"]) and (cur["close"] > cur["ema_fast"])
    downtrend = (cur["ema_fast"] < cur["ema_slow"]) and (cur["close"] < cur["ema_fast"])

    pullback_long = (prev["close"] < prev["ema_fast"]) and (cur["close"] > cur["ema_fast"])
    pullback_short = (prev["close"] > prev["ema_fast"]) and (cur["close"] < cur["ema_fast"])

    macd_turns_up = (cur["macd_hist"] > prev["macd_hist"]) and (prev["macd_hist"] > prev2["macd_hist"])
    macd_turns_down = (cur["macd_hist"] < prev["macd_hist"]) and (prev["macd_hist"] < prev2["macd_hist"])

    near_mid_long = cur["close"] >= cur["bb_mid"]
    near_mid_short = cur["close"] <= cur["bb_mid"]

    if uptrend and pullback_long and macd_turns_up and near_mid_long:
        return "LONG_PULLBACK_CONTINUATION"

    if downtrend and pullback_short and macd_turns_down and near_mid_short:
        return "SHORT_PULLBACK_CONTINUATION"

    return "WAIT"


# =========================
# TRADE PLAN (ENTRY/SL/TP)
# =========================

def recent_swing_low(df: pd.DataFrame, lookback: int) -> Optional[float]:
    if df.empty or len(df) < lookback:
        return None
    return float(df.iloc[-lookback:]["low"].min())


def recent_swing_high(df: pd.DataFrame, lookback: int) -> Optional[float]:
    if df.empty or len(df) < lookback:
        return None
    return float(df.iloc[-lookback:]["high"].max())


def make_trade_plan(df_ind: pd.DataFrame, cfg: StrategyConfig, label: str) -> Optional[TradePlan]:
    if df_ind.empty:
        return None

    cur = df_ind.iloc[-1]
    entry = float(cur["close"])

    if label == "LONG_PULLBACK_CONTINUATION":
        base_sl = recent_swing_low(df_ind, cfg.swing_lookback)
        if base_sl is None and pd.notna(cur.get("bb_lower", None)):
            base_sl = float(cur["bb_lower"])
        if base_sl is None:
            return None

        sl = float(base_sl) - float(cfg.sl_buffer)
        risk = entry - sl
        if risk <= 0:
            return None

        tp = entry + cfg.rr * risk
        rationale = (
            f"LPC LONG: uptrend + pullback->ema_fast + MACD_hist omhoog. "
            f"SL swing-low({cfg.swing_lookback})+buffer. TP {cfg.rr:.2f}R."
        )
        return TradePlan(side="LONG", entry=entry, sl=sl, tp=tp, rationale=rationale)

    if label == "SHORT_PULLBACK_CONTINUATION":
        base_sl = recent_swing_high(df_ind, cfg.swing_lookback)
        if base_sl is None and pd.notna(cur.get("bb_upper", None)):
            base_sl = float(cur["bb_upper"])
        if base_sl is None:
            return None

        sl = float(base_sl) + float(cfg.sl_buffer)
        risk = sl - entry
        if risk <= 0:
            return None

        tp = entry - cfg.rr * risk
        rationale = (
            f"LPC SHORT: downtrend + pullback->ema_fast + MACD_hist omlaag. "
            f"SL swing-high({cfg.swing_lookback})+buffer. TP {cfg.rr:.2f}R."
        )
        return TradePlan(side="SHORT", entry=entry, sl=sl, tp=tp, rationale=rationale)

    return None


# =========================
# ADVICE COMPUTATION
# =========================

def compute_advice_from_db(
    db_path: str,
    symbol: str,
    timeframe: str,
    cfg: StrategyConfig,
    limit_rows: int = 20000
) -> AdviceResult:
    now = datetime.now(timezone.utc).isoformat(timespec="seconds")

    prices = load_prices_from_sqlite(db_path=db_path, limit_rows=limit_rows)
    ohlc = prices_to_ohlc(prices, timeframe=timeframe)

    if ohlc.empty:
        return AdviceResult(label="NO_DATA", message=f"{now} | {symbol} | NO_DATA (geen OHLC)", ts_utc=now)

    df_ind = compute_indicators(ohlc, cfg)
    label = label_latest(df_ind, cfg)
    plan = make_trade_plan(df_ind, cfg, label)

    last = ohlc.iloc[-1]
    base_msg = (
        f"{now} | {symbol} | tf={timeframe}\n"
        f"Last bar (UTC): {ohlc.index[-1].isoformat()}\n"
        f"O/H/L/C: {last['open']:.5f} / {last['high']:.5f} / {last['low']:.5f} / {last['close']:.5f}\n"
        f"LABEL: {label}"
    )

    if plan:
        plan_msg = (
            "\n\nTRADE PLAN\n"
            f"Side: {plan.side}\n"
            f"Entry: {plan.entry:.5f}\n"
            f"SL: {plan.sl:.5f}\n"
            f"TP: {plan.tp:.5f}\n"
            f"Note: {plan.rationale}"
        )
        return AdviceResult(label=label, message=base_msg + plan_msg, ts_utc=now, plan=plan)

    return AdviceResult(label=label, message=base_msg, ts_utc=now, plan=None)


# =========================
# PROWL (PUSH)
# =========================

def read_prowl_key(path: str) -> str:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Prowl key bestand niet gevonden: {path}")
    with open(path, "r", encoding="utf-8") as f:
        key = f.read().strip()
    if not key:
        raise ValueError("Prowl key is leeg.")
    return key


def prowl_send(apikey: str, application: str, event: str, description: str, priority: int = 0) -> None:
    url = "https://api.prowlapp.com/publicapi/add"
    data = {
        "apikey": apikey,
        "application": application,
        "event": event,
        "description": description,
        "priority": str(priority),
    }
    r = requests.post(url, data=data, timeout=15)
    if r.status_code != 200:
        raise RuntimeError(f"Prowl error {r.status_code}: {r.text[:200]}")


# =========================
# STATE (dedupe) in SQLite
# =========================

def _ensure_state_table(conn: sqlite3.Connection) -> None:
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS monitor_state (
            k TEXT PRIMARY KEY,
            v TEXT,
            updated_utc TEXT
        )
    """)
    conn.commit()


def get_state(db_path: str, key: str) -> Optional[str]:
    conn = sqlite3.connect(db_path)
    try:
        _ensure_state_table(conn)
        cur = conn.cursor()
        row = cur.execute("SELECT v FROM monitor_state WHERE k = ?", (key,)).fetchone()
        return row[0] if row else None
    finally:
        conn.close()


def set_state(db_path: str, key: str, value: str) -> None:
    conn = sqlite3.connect(db_path)
    try:
        _ensure_state_table(conn)
        cur = conn.cursor()
        now = datetime.now(timezone.utc).isoformat(timespec="seconds")
        cur.execute("""
            INSERT INTO monitor_state (k, v, updated_utc)
            VALUES (?, ?, ?)
            ON CONFLICT(k) DO UPDATE SET v=excluded.v, updated_utc=excluded.updated_utc
        """, (key, value, now))
        conn.commit()
    finally:
        conn.close()


# =========================
# TRADE JOURNAL in SQLite
# =========================

def _ensure_trade_journal_table(conn: sqlite3.Connection) -> None:
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS trade_journal (
            trade_id   TEXT PRIMARY KEY,
            opened_utc TEXT NOT NULL,
            closed_utc TEXT,
            symbol     TEXT,
            timeframe  TEXT,
            side       TEXT,
            entry      REAL,
            sl         REAL,
            tp         REAL,
            exit       REAL,
            outcome    TEXT,      -- WIN / LOSS / UNKNOWN
            pnl_pct    REAL,
            r_multiple REAL,
            note       TEXT
        )
    """)
    cur.execute("CREATE INDEX IF NOT EXISTS idx_trade_journal_closed ON trade_journal(closed_utc)")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_trade_journal_symbol_tf ON trade_journal(symbol, timeframe)")
    conn.commit()


def _dt_utc_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")


def open_trade_journal(
    db_path: str,
    symbol: str,
    timeframe: str,
    plan: TradePlan,
    note: str = ""
) -> str:
    trade_id = f"{symbol.upper()}::{timeframe}::{plan.plan_id()}::{_dt_utc_iso()}"
    conn = sqlite3.connect(db_path)
    try:
        _ensure_trade_journal_table(conn)
        cur = conn.cursor()
        cur.execute("""
            INSERT INTO trade_journal (
                trade_id, opened_utc, symbol, timeframe, side, entry, sl, tp, note
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            trade_id, _dt_utc_iso(), symbol.upper(), timeframe, plan.side,
            float(plan.entry), float(plan.sl), float(plan.tp), note
        ))
        conn.commit()
        return trade_id
    finally:
        conn.close()


def get_open_trade(db_path: str, symbol: str, timeframe: str) -> Optional[Dict[str, Any]]:
    conn = sqlite3.connect(db_path)
    try:
        _ensure_trade_journal_table(conn)
        cur = conn.cursor()
        row = cur.execute("""
            SELECT trade_id, opened_utc, side, entry, sl, tp
            FROM trade_journal
            WHERE closed_utc IS NULL AND symbol = ? AND timeframe = ?
            ORDER BY opened_utc DESC
            LIMIT 1
        """, (symbol.upper(), timeframe)).fetchone()
        if not row:
            return None
        return {
            "trade_id": row[0],
            "opened_utc": row[1],
            "side": row[2],
            "entry": float(row[3]),
            "sl": float(row[4]),
            "tp": float(row[5]),
        }
    finally:
        conn.close()


def close_trade_journal(
    db_path: str,
    trade_id: str,
    exit_price: float,
    outcome: str,
    pnl_pct: float,
    r_multiple: float,
    note: str = ""
) -> None:
    conn = sqlite3.connect(db_path)
    try:
        _ensure_trade_journal_table(conn)
        cur = conn.cursor()
        cur.execute("""
            UPDATE trade_journal
            SET closed_utc = ?, exit = ?, outcome = ?, pnl_pct = ?, r_multiple = ?,
                note = COALESCE(note,'') || ?
            WHERE trade_id = ?
        """, (_dt_utc_iso(), float(exit_price), outcome, float(pnl_pct), float(r_multiple),
              ("\n" + note if note else ""), trade_id))
        conn.commit()
    finally:
        conn.close()


def compute_trade_stats(db_path: str, symbol: str, timeframe: str) -> Dict[str, Any]:
    conn = sqlite3.connect(db_path)
    try:
        _ensure_trade_journal_table(conn)
        cur = conn.cursor()
        rows = cur.execute("""
            SELECT outcome, pnl_pct
            FROM trade_journal
            WHERE closed_utc IS NOT NULL AND symbol = ? AND timeframe = ?
        """, (symbol.upper(), timeframe)).fetchall()

        total = len(rows)
        wins = sum(1 for o, _ in rows if o == "WIN")
        losses = sum(1 for o, _ in rows if o == "LOSS")

        pnl_vals = [float(p) for _, p in rows if p is not None]
        sum_pnl = sum(pnl_vals) if pnl_vals else 0.0
        avg_pnl = (sum_pnl / len(pnl_vals)) if pnl_vals else 0.0

        gross_profit = sum(float(p) for o, p in rows if o == "WIN" and p is not None and float(p) > 0)
        gross_loss = sum(float(p) for o, p in rows if o == "LOSS" and p is not None and float(p) < 0)
        profit_factor = (gross_profit / abs(gross_loss)) if gross_loss < 0 else (float("inf") if gross_profit > 0 else 0.0)

        winrate = (wins / total * 100.0) if total else 0.0
        lossrate = (losses / total * 100.0) if total else 0.0

        return {
            "total": total,
            "wins": wins,
            "losses": losses,
            "winrate": winrate,
            "lossrate": lossrate,
            "sum_pnl_pct": sum_pnl,
            "avg_pnl_pct": avg_pnl,
            "profit_factor": profit_factor,
        }
    finally:
        conn.close()


def last_closed_trades_summary(db_path: str, symbol: str, timeframe: str, n: int = 10) -> str:
    conn = sqlite3.connect(db_path)
    try:
        _ensure_trade_journal_table(conn)
        cur = conn.cursor()
        rows = cur.execute("""
            SELECT opened_utc, closed_utc, side, outcome, pnl_pct
            FROM trade_journal
            WHERE closed_utc IS NOT NULL AND symbol = ? AND timeframe = ?
            ORDER BY closed_utc DESC
            LIMIT ?
        """, (symbol.upper(), timeframe, n)).fetchall()

        if not rows:
            return "Geen gesloten trades in journal."

        lines = []
        for opened_utc, closed_utc, side, outcome, pnl_pct in rows:
            p = float(pnl_pct) if pnl_pct is not None else 0.0
            lines.append(f"- {closed_utc} | {side} | {outcome} | {p:+.2f}% (open {opened_utc})")
        return "\n".join(lines)
    finally:
        conn.close()


def _iso_to_dt(s: str) -> datetime:
    return datetime.fromisoformat(s).astimezone(timezone.utc)


def evaluate_open_trade_on_prices(open_trade: Dict[str, Any], prices_df: pd.DataFrame) -> Optional[Dict[str, Any]]:
    if prices_df.empty:
        return None

    opened_dt = _iso_to_dt(open_trade["opened_utc"])
    df = prices_df.copy()
    df = df[df["ts"] > opened_dt].sort_values("ts")
    if df.empty:
        return None

    side = str(open_trade["side"]).upper()
    entry = float(open_trade["entry"])
    sl = float(open_trade["sl"])
    tp = float(open_trade["tp"])

    for _, r in df.iterrows():
        px = float(r["price"])

        if side == "LONG":
            tp_hit = px >= tp
            sl_hit = px <= sl
            if tp_hit and sl_hit:
                hit, exit_px = "LOSS", sl
            elif sl_hit:
                hit, exit_px = "LOSS", sl
            elif tp_hit:
                hit, exit_px = "WIN", tp
            else:
                continue

            pnl_pct = (exit_px - entry) / entry * 100.0
            risk = entry - sl
            r_mult = (exit_px - entry) / risk if risk != 0 else 0.0
            return {"outcome": hit, "exit": exit_px, "pnl_pct": pnl_pct, "r_multiple": r_mult}

        if side == "SHORT":
            tp_hit = px <= tp
            sl_hit = px >= sl
            if tp_hit and sl_hit:
                hit, exit_px = "LOSS", sl
            elif sl_hit:
                hit, exit_px = "LOSS", sl
            elif tp_hit:
                hit, exit_px = "WIN", tp
            else:
                continue

            pnl_pct = (entry - exit_px) / entry * 100.0
            risk = sl - entry
            r_mult = (entry - exit_px) / risk if risk != 0 else 0.0
            return {"outcome": hit, "exit": exit_px, "pnl_pct": pnl_pct, "r_multiple": r_mult}

    return None


def manual_close_open_trade(
    db_path: str,
    symbol: str,
    timeframe: str,
    exit_price: float,
    reason: str = "Manual close"
) -> Optional[Dict[str, Any]]:
    """
    Sluit de huidige open trade en retourneert een dict met trade_id/outcome/pnl_pct/r_multiple.
    """
    open_tr = get_open_trade(db_path, symbol, timeframe)
    if open_tr is None:
        return None

    side = str(open_tr["side"]).upper()
    entry = float(open_tr["entry"])
    sl = float(open_tr["sl"])

    if side == "LONG":
        pnl_pct = (exit_price - entry) / entry * 100.0
        risk = entry - sl
        r_mult = (exit_price - entry) / risk if risk != 0 else 0.0
    else:  # SHORT
        pnl_pct = (entry - exit_price) / entry * 100.0
        risk = sl - entry
        r_mult = (entry - exit_price) / risk if risk != 0 else 0.0

    outcome = "WIN" if pnl_pct > 0 else ("LOSS" if pnl_pct < 0 else "UNKNOWN")

    close_trade_journal(
        db_path=db_path,
        trade_id=open_tr["trade_id"],
        exit_price=float(exit_price),
        outcome=outcome,
        pnl_pct=float(pnl_pct),
        r_multiple=float(r_mult),
        note=reason
    )

    return {
        "trade_id": open_tr["trade_id"],
        "outcome": outcome,
        "exit": float(exit_price),
        "pnl_pct": float(pnl_pct),
        "r_multiple": float(r_mult),
    }


# =========================
# MONITOR THREAD (manual start from notebook)
# =========================

_stop = threading.Event()
_thread: Optional[threading.Thread] = None


def start_monitor(
    key_path: str = "key.txt",
    interval_sec: int = 60,
    symbol: str = "XAUUSD",
    timeframe: str = "1min",
    limit_rows: int = 20000,
    prowl_app: str = "Trade Strategy",
    prowl_priority: int = 0,
    notify_on_start: bool = False,
    state_key: Optional[str] = None,
    cfg: Optional[StrategyConfig] = None,
    db_path: Optional[str] = None,
) -> None:
    """
    Start de monitor thread.
    - Als er een open trade is: monitort TP/SL en pusht RESULT bij close
    - Als er geen open trade is: zoekt setup en pusht TRADE bij nieuw plan
    """
    global _thread
    if _thread is not None and _thread.is_alive():
        print("[monitor] draait al")
        return

    db_path = db_path or db_path_from_env()
    apikey = read_prowl_key(key_path)
    cfg = cfg or StrategyConfig()
    state_key = state_key or f"last_sent_plan::{symbol.upper()}::{timeframe}"

    _stop.clear()

    def worker():
        print(f"[monitor] gestart | DB={db_path} | {symbol} | tf={timeframe} | interval={interval_sec}s")

        # Ensure tables exist
        conn = sqlite3.connect(db_path)
        try:
            _ensure_state_table(conn)
            _ensure_trade_journal_table(conn)
        finally:
            conn.close()

        # Init state
        try:
            res0 = compute_advice_from_db(db_path, symbol, timeframe, cfg, limit_rows=limit_rows)
            if res0.plan is not None:
                pid0 = res0.plan.plan_id()
                if get_state(db_path, state_key) is None:
                    set_state(db_path, state_key, pid0)

                if notify_on_start and get_open_trade(db_path, symbol, timeframe) is None:
                    trade_id = open_trade_journal(
                        db_path=db_path,
                        symbol=symbol,
                        timeframe=timeframe,
                        plan=res0.plan,
                        note=f"Setup label={res0.label} (notify_on_start=True)"
                    )
                    prowl_send(
                        apikey=apikey,
                        application=prowl_app,
                        event=f"TRADE: {symbol} {timeframe} {res0.plan.side}",
                        description=res0.message + f"\n\nJournal trade_id: {trade_id}",
                        priority=prowl_priority,
                    )
                    print(f"[monitor] PROWL sent (start) | opened {trade_id}")
        except Exception as e:
            print("[monitor] init ERROR:", e)

        while not _stop.is_set():
            try:
                # 1) Open trade? monitor TP/SL
                open_tr = get_open_trade(db_path, symbol, timeframe)
                recent_prices = load_prices_from_sqlite(db_path=db_path, limit_rows=min(limit_rows, 8000))

                if open_tr is not None:
                    closed = evaluate_open_trade_on_prices(open_tr, recent_prices)
                    if closed is not None:
                        close_trade_journal(
                            db_path=db_path,
                            trade_id=open_tr["trade_id"],
                            exit_price=closed["exit"],
                            outcome=closed["outcome"],
                            pnl_pct=closed["pnl_pct"],
                            r_multiple=closed["r_multiple"],
                            note="Auto-close via TP/SL check op price stream."
                        )

                        stats = compute_trade_stats(db_path, symbol, timeframe)
                        history = last_closed_trades_summary(db_path, symbol, timeframe, n=10)

                        msg = (
                            f"{_dt_utc_iso()} | {symbol} | tf={timeframe}\n"
                            f"TRADE CLOSED: {closed['outcome']}\n"
                            f"Side: {open_tr['side']}\n"
                            f"Entry: {open_tr['entry']:.5f}\n"
                            f"SL: {open_tr['sl']:.5f}\n"
                            f"TP: {open_tr['tp']:.5f}\n"
                            f"Exit: {closed['exit']:.5f}\n"
                            f"PnL: {closed['pnl_pct']:+.2f}% | R: {closed['r_multiple']:+.2f}\n\n"
                            f"STATS (closed trades)\n"
                            f"Total: {stats['total']} | Wins: {stats['wins']} | Losses: {stats['losses']}\n"
                            f"Winrate: {stats['winrate']:.1f}% | Lossrate: {stats['lossrate']:.1f}%\n"
                            f"Sum PnL: {stats['sum_pnl_pct']:+.2f}% | Avg PnL: {stats['avg_pnl_pct']:+.2f}%\n"
                            f"Profit factor: {stats['profit_factor']:.2f}\n\n"
                            f"LAST TRADES\n{history}"
                        )

                        prowl_send(
                            apikey=apikey,
                            application=prowl_app,
                            event=f"RESULT: {symbol} {timeframe} {closed['outcome']}",
                            description=msg,
                            priority=prowl_priority,
                        )
                        print(f"[monitor] PROWL result sent | {open_tr['trade_id']} | {closed['outcome']}")

                        _stop.wait(interval_sec)
                        continue

                    print("[monitor] trade open | monitoring TP/SL...")
                    _stop.wait(interval_sec)
                    continue

                # 2) Geen open trade: compute setup en push bij nieuw plan
                res = compute_advice_from_db(db_path, symbol, timeframe, cfg, limit_rows=limit_rows)

                if res.plan is None:
                    print(f"[monitor] no trade | {res.label}")
                else:
                    plan_id = res.plan.plan_id()
                    last_sent = get_state(db_path, state_key)

                    if last_sent != plan_id:
                        set_state(db_path, state_key, plan_id)

                        trade_id = open_trade_journal(
                            db_path=db_path,
                            symbol=symbol,
                            timeframe=timeframe,
                            plan=res.plan,
                            note=f"Setup label={res.label}"
                        )

                        prowl_send(
                            apikey=apikey,
                            application=prowl_app,
                            event=f"TRADE: {symbol} {timeframe} {res.plan.side}",
                            description=res.message + f"\n\nJournal trade_id: {trade_id}",
                            priority=prowl_priority,
                        )
                        print(f"[monitor] PROWL sent | opened {trade_id}")
                    else:
                        print(f"[monitor] same plan | {plan_id} (no push)")

            except Exception as e:
                print("[monitor] ERROR:", e)

            _stop.wait(interval_sec)

    _thread = threading.Thread(target=worker, daemon=True)
    _thread.start()


def stop_monitor() -> None:
    _stop.set()
    print("[monitor] stop signaal gestuurd")


def monitor_is_running() -> bool:
    return _thread is not None and _thread.is_alive()


if __name__ == "__main__":
    # Expliciet: NIETS automatisch starten.
    print("Dit is een module. Start de monitor vanuit je notebook met start_monitor(...).")


In [44]:
stop_monitor()

start_monitor(
        key_path="key.txt",
        interval_sec=60,
        symbol="XAUUSD",
        timeframe="1min",
        limit_rows=20000,
        prowl_app="Trade Strategy",
        prowl_priority=0,
        notify_on_start=False,
        cfg=cfg,
        db_path="gold_prices.sqlite",  # <-- wijzig dit indien jouw DB anders heet
    )


[monitor] stop signaal gestuurd
[monitor] gestart | DB=gold_prices.sqlite | XAUUSD | tf=1min | interval=60s


[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] no trade | WAIT
[monitor] PROWL sent | opened XAUUSD::1min::LONG|4422.89990|4414.70020|4435.19946::2025-12-22T13:33:15+00:00
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] trade open | monitoring TP/SL...
[monitor] t

In [ ]:
import sqlite3

def delete_last_open_trade(db_path: str, symbol: str, timeframe: str):
    conn = sqlite3.connect(db_path)
    try:
        cur = conn.cursor()

        row = cur.execute("""
            SELECT trade_id, opened_utc, side, entry, sl, tp, closed_utc
            FROM trade_journal
            WHERE symbol = ? AND timeframe = ? AND closed_utc IS NULL
            ORDER BY opened_utc DESC
            LIMIT 1
        """, (symbol.upper(), timeframe)).fetchone()

        if not row:
            print("Geen OPEN trade gevonden om te verwijderen.")
            return None

        trade_id = row[0]
        cur.execute("DELETE FROM trade_journal WHERE trade_id = ?", (trade_id,))
        conn.commit()

        print("Verwijderd (OPEN trade):", trade_id)
        print("Details:", row)
        return trade_id
    finally:
        conn.close()

# Gebruik:
DB_PATH = "gold_prices.sqlite"   # pas aan
SYMBOL = "XAUUSD"           # pas aan
TIMEFRAME = "1min"          # pas aan

delete_last_open_trade(DB_PATH, SYMBOL, TIMEFRAME)



